In [120]:
import pymysql
import pymysql.cursors
import sqlite3
import ipywidgets as widgets
from IPython.display import display
import datetime, numpy

In [121]:
def get_db_conn_root():
    connection = pymysql.connect(
                    host='127.0.0.1',
                    user='root',
                    password='x3n0n136',
                    db='lab206',
                    port=3308,
                    cursorclass=pymysql.cursors.DictCursor)
    return connection

def get_db_conn_write():
    connection = pymysql.connect(
                    host='127.0.0.1',
                    user='exo_rw',
                    password='x3n0n',
                    db='lab206',
                    port=3308,
                    cursorclass=pymysql.cursors.DictCursor)
    return connection

def get_db_conn_read():
    connection = pymysql.connect(
                    host='127.0.0.1',
                    user='exo_r0',
                    password='xenon',
                    db='lab206',
                    port=3308,
                    cursorclass=pymysql.cursors.DictCursor)
    return connection


def setup_db():
    connection = get_db_conn_root()
    haz_exoro = False
    haz_exorw = False
    haz_table = False
    #Check Users:
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT User from mysql.user")
            ret_users = cursor.fetchall()
            cursor.execute("SHOW TABLES LIKE 'testtable'")
            ret_tables = cursor.fetchall()
    finally:
        connection.close()
    for u in ret_users:
        if u['User'] == 'exo_ro':
            print("User exo_ro already exists")
            haz_exoro = True
        elif u['User'] == 'exo_rw':
            print("User exo_rw already exists")
            haz_exorw = True
    if len(ret_tables)>0:
            print("Table testtables already exists")
            haz_table = True
    if False in [haz_exoro, haz_exorw, haz_table]:
        connection = get_db_conn_root()
        try:
            with connection.cursor() as cursor:
                if not haz_exoro:
                    print("Creating exo_ro user")
                    sql = "CREATE USER 'exo_ro'@'%' IDENTIFIED BY 'xenon';"
                    cursor.execute(sql)
                    connection.commit()
                    sql = "GRANT SELECT ON lab206.* to 'exo_ro'@'%';"
                    cursor.execute(sql)
                    connection.commit()
                    cursor.execute("FLUSH PRIVILEGES")
                    connection.commit()
                if not haz_exorw:
                    print("Creating exo_rw user")
                    sql = "CREATE USER 'exo_rw'@'%' IDENTIFIED BY 'x3n0n';"
                    cursor.execute(sql)
                    connection.commit()
                    sql = "GRANT SELECT, INSERT ON lab206.* to 'exo_rw'@'%'"
                    cursor.execute(sql)
                    connection.commit()
                    cursor.execute("FLUSH PRIVILEGES")
                    connection.commit()
                if not haz_table:
                    print("Creating table testtable")
                    sql = """CREATE TABLE testtable (
                             time TIMESTAMP, 
                             v1 DECIMAL(6,4),
                             v2 DECIMAL(6,4),
                             v3 DECIMAL(6,4),
                             v4 DECIMAL(6,4),
                             v5 DECIMAL(6,4)
                             );"""
                    cursor.execute(sql)
                    connection.commit()
        finally:
            connection.close()   
            
def get_db_users():
    connection = get_db_conn_root()
    try:
        with connection.cursor() as cursor:
            sql = "SELECT User, Host, authentication_string FROM mysql.user;"
            cursor.execute(sql)
            users = cursor.fetchall()
    finally:
        connection.close()
    print(users)

In [122]:
setup_db()
get_db_users()

Creating exo_ro user
Creating exo_rw user
Creating table testtable
[{'User': 'exo_ro', 'Host': '%', 'authentication_string': '$A$005$(P+(K(/c{Ar\x0c\\?\x04\x0f\x1c99\x18jxWA2dsZzNDsi0YDcBlJtn9vQhflGr7mrJGpABtYQtC'}, {'User': 'exo_rw', 'Host': '%', 'authentication_string': '$A$005$\x05/qmQ\x01H{\x1b2UE\x1a0[U\x04\x7f\x02}YLFJTrn2HimAPDsKherZGzPFCviPf5LRO17iBR2.b44'}, {'User': 'root', 'Host': '%', 'authentication_string': '$A$005$YeP\x02*\x01\\p>>c2kO\x07\x083Cm*BPZGYdprkffOHDXofWWOpqMgckV28Nt6mEUi10fYcPA'}, {'User': 'mysql.infoschema', 'Host': 'localhost', 'authentication_string': '$A$005$THISISACOMBINATIONOFINVALIDSALTANDPASSWORDTHATMUSTNEVERBRBEUSED'}, {'User': 'mysql.session', 'Host': 'localhost', 'authentication_string': '$A$005$THISISACOMBINATIONOFINVALIDSALTANDPASSWORDTHATMUSTNEVERBRBEUSED'}, {'User': 'mysql.sys', 'Host': 'localhost', 'authentication_string': '$A$005$THISISACOMBINATIONOFINVALIDSALTANDPASSWORDTHATMUSTNEVERBRBEUSED'}, {'User': 'root', 'Host': 'localhost', 'authentic

In [123]:
def db_data_point():
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    #timestamp = time.time()
    randdata = numpy.random.random_sample(5)
    v1 = 25+randdata[0]
    v2 = 25+randdata[1]*5
    v3 = 25+randdata[2]*3
    v4 = 1+randdata[3]*0.25
    v5 = 5+randdata[4]
    connection = get_db_conn_write()
    try:
        with connection.cursor() as cursor:
            values_str = "'%s', '%f', '%f', '%f', '%.4f', '%f'"%(timestamp, v1, v2, v3, v4, v5)
            sql = """INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
                     VALUES (%s);"""%values_str
            print(sql)
            cursor.execute(sql)
            connection.commit()
    finally:
        connection.close()            

In [ ]:
for i in range(100):
    db_data_point()
    time.sleep(2)

INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
                     VALUES ('2020-01-19 18:34:46', '25.889077', '28.463496', '27.153654', '1.0287', '5.417481');


C:\Users\bung\Anaconda3\envs\mysqlplotly\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'v1' at row 1")
  self._do_get_result()
C:\Users\bung\Anaconda3\envs\mysqlplotly\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'v2' at row 1")
  self._do_get_result()
C:\Users\bung\Anaconda3\envs\mysqlplotly\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'v3' at row 1")
  self._do_get_result()
C:\Users\bung\Anaconda3\envs\mysqlplotly\lib\site-packages\pymysql\cursors.py:329: Warning: (1265, "Data truncated for column 'v5' at row 1")
  self._do_get_result()


INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
                     VALUES ('2020-01-19 18:34:48', '25.722996', '28.366987', '27.335945', '1.1027', '5.715816');
INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
                     VALUES ('2020-01-19 18:34:50', '25.519226', '27.719219', '27.132911', '1.1621', '5.158477');
INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
                     VALUES ('2020-01-19 18:34:52', '25.980271', '28.203535', '25.058357', '1.2097', '5.299559');
INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
                     VALUES ('2020-01-19 18:34:54', '25.058401', '25.419512', '26.335637', '1.1791', '5.862456');
INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
                     VALUES ('2020-01-19 18:34:56', '25.985767', '28.586938', '27.406951', '1.2392', '5.746111');
INSERT INTO testtable 
                     (time, v1, v2, v3, v4, v5)
    

1579485222.818773